RNN using PyTorch | Question Answering System using PyTorch | Video 13 | CampusX

https://www.youtube.com/watch?v=xjzWrPQ66VQ&list=PLKnIA16_Rmvboy8bmDCjwNHgTaYH2puK7&index=13

In [53]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv("../Dataset/100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [55]:
# Tokenize Words 
def tokenize(text):
  text = text.lower()
  text = text.replace("?","")
  text = text.replace("'","")
  return text.split()

tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

In [56]:
# Create A words Vocabulari (find total unique words in dataset)
# first create a dictionay so we can store each unique word with it index

vocab = {'<UNK>':0}

def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [57]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [58]:
print(vocab)

{'<UNK>': 0, 'what': 1, 'is': 2, 'the': 3, 'capital': 4, 'of': 5, 'france': 6, 'paris': 7, 'germany': 8, 'berlin': 9, 'who': 10, 'wrote': 11, 'to': 12, 'kill': 13, 'a': 14, 'mockingbird': 15, 'harper-lee': 16, 'largest': 17, 'planet': 18, 'in': 19, 'our': 20, 'solar': 21, 'system': 22, 'jupiter': 23, 'boiling': 24, 'point': 25, 'water': 26, 'celsius': 27, '100': 28, 'painted': 29, 'mona': 30, 'lisa': 31, 'leonardo-da-vinci': 32, 'square': 33, 'root': 34, '64': 35, '8': 36, 'chemical': 37, 'symbol': 38, 'for': 39, 'gold': 40, 'au': 41, 'which': 42, 'year': 43, 'did': 44, 'world': 45, 'war': 46, 'ii': 47, 'end': 48, '1945': 49, 'longest': 50, 'river': 51, 'nile': 52, 'japan': 53, 'tokyo': 54, 'developed': 55, 'theory': 56, 'relativity': 57, 'albert-einstein': 58, 'freezing': 59, 'fahrenheit': 60, '32': 61, 'known': 62, 'as': 63, 'red': 64, 'mars': 65, 'author': 66, '1984': 67, 'george-orwell': 68, 'currency': 69, 'united': 70, 'kingdom': 71, 'pound': 72, 'india': 73, 'delhi': 74, 'discov

In [59]:
len(vocab)

324

In [60]:
# convert words to numerical indices

def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  
  return indexed_text

In [61]:
text_to_indices("france", vocab)

[6]

In [62]:
import torch
from torch.utils.data import Dataset, DataLoader

In [63]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]
  
  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [64]:
dataset = QADataset(df, vocab)

In [65]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [83]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [84]:
for question, answer in dataloader:
  print(question, answer)

tensor([[10, 11, 12, 13, 14, 15]]) tensor([[16]])
tensor([[78, 79, 80, 81, 82, 83, 84]]) tensor([[85]])
tensor([[1, 2, 3, 4, 5, 8]]) tensor([[9]])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([[287]])
tensor([[ 42, 107,   2, 108,  19, 109]]) tensor([[110]])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([[173]])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([[166]])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([[113]])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([[128]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([[91]])
tensor([[  1,   2,   3, 141, 117,  83,   3, 277, 278]]) tensor([[121]])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([[233]])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([[149]])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([[273]])
tensor([[ 42, 101,   2,   3,  17]

## Now Design Our RNN Architecture

In [85]:
import torch.nn as nn

In [86]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [87]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [93]:
learning_rate = 0.001
epochs = 60

In [94]:
model = SimpleRNN(len(vocab))

In [95]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [96]:
import torch
from torch.utils.data import Dataset, DataLoader

In [97]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 521.289861
Epoch: 2, Loss: 449.226931
Epoch: 3, Loss: 372.449696
Epoch: 4, Loss: 314.276651
Epoch: 5, Loss: 264.359702
Epoch: 6, Loss: 216.572748
Epoch: 7, Loss: 172.396774
Epoch: 8, Loss: 134.442740
Epoch: 9, Loss: 103.066869
Epoch: 10, Loss: 78.863242
Epoch: 11, Loss: 60.929637
Epoch: 12, Loss: 47.640117
Epoch: 13, Loss: 37.761083
Epoch: 14, Loss: 30.687868
Epoch: 15, Loss: 25.273892
Epoch: 16, Loss: 21.259196
Epoch: 17, Loss: 18.100846
Epoch: 18, Loss: 15.408927
Epoch: 19, Loss: 13.383157
Epoch: 20, Loss: 11.604631
Epoch: 21, Loss: 10.264140
Epoch: 22, Loss: 9.012790
Epoch: 23, Loss: 7.933586
Epoch: 24, Loss: 7.114678
Epoch: 25, Loss: 6.353780
Epoch: 26, Loss: 5.740671
Epoch: 27, Loss: 5.186834
Epoch: 28, Loss: 4.701916
Epoch: 29, Loss: 4.291141
Epoch: 30, Loss: 3.916531
Epoch: 31, Loss: 3.590623
Epoch: 32, Loss: 3.299019
Epoch: 33, Loss: 3.047155
Epoch: 34, Loss: 2.815495
Epoch: 35, Loss: 2.610094
Epoch: 36, Loss: 2.417628
Epoch: 37, Loss: 2.252792
Epoch: 38, Loss: 

In [ ]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value <  threshold: 
    print("I dont Know")

  print(list(vocab.keys())[index])

In [112]:
user_input = input("Enter Your Question: ")
predict(model, user_input)

berlin
